In [1]:
import numpy as np
import cv2
import os
import bck_adq
from motpy import Detection, MultiObjectTracker
from csv import writer 
import pandas as pd
from deep_sort_realtime.deepsort_tracker import DeepSort
import sys

c:\Users\ccana\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\ccana\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\ccana\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


ModuleNotFoundError: No module named 'motpy'

In [2]:
def fixColor(image):
    return(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

In [6]:
%cd C:/Users/ccana/Documents/Kitai/KITAI_v2.0/muestras

C:\Users\ccana\Documents\Kitai\KITAI_v2.0\muestras


In [7]:
#path='trial.mp4'
path='7211454.mp4'
video_stream= cv2.VideoCapture(path)
total_frames=video_stream.get(cv2.CAP_PROP_FRAME_COUNT)

In [8]:
total_frames

52389.0

In [21]:
#Create folder for the result
folderpath=path.split('.')[0]+' Dataset'
if not os.path.isdir(folderpath):
   os.makedirs(folderpath)

In [6]:
# Putting in together for processing video
#writer = cv2.VideoWriter(os.path.join(folderpath,"output.avi"),cv2.VideoWriter_fourcc('M','J','P','G'), 30,(640,480))

In [22]:
grayMedianFrame= bck_adq.grayFrame(path)

In [23]:
model_spec = {
        'order_pos': 1, 'dim_pos': 2, # position is a center in 2D space; under constant velocity model
        'order_size': 0, 'dim_size': 2, # bounding box is 2 dimensional; under constant velocity model
        'q_var_pos': 1000., # process noise
        'r_var_pos': 0.1 # measurement noise
    }
tracker = MultiObjectTracker(dt=0.1)

In [24]:
frameCnt=0
data=[]
try:
    while(frameCnt < total_frames-1):
        ret, frame = video_stream.read()
        #% of conversion
        frameCnt+=1
        print('Process: '+str(frameCnt/total_frames*100)+'% ',end="\r")

        # Convert current frame to grayscale
        gframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Calculate absolute difference of current frame and the median frame
        dframe = cv2.absdiff(gframe, grayMedianFrame)

        # Gaussian
        blurred = cv2.GaussianBlur(dframe, (11, 11), 0)

        #Thresholding to binarise
        ret, tframe= cv2.threshold(blurred,0,255,
                                   cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        #Identifying contours from the threshold
        (cnts, _) = cv2.findContours(tframe.copy(), 
                                     cv2.RETR_EXTERNAL, cv2 .CHAIN_APPROX_SIMPLE)


        if len(cnts) < 100:
            #For each contour draw the bounding bos
            for cnt in cnts:
                x,y,w,h = cv2.boundingRect(cnt)
                tracker.step(detections=[Detection(box=np.array([int(x-w),int(y-h),int(x+2*w),int(y+2*h)]))])
                tracks = tracker.active_tracks()
                #cv2.rectangle(frame,(int(x-w),int(y-h)),(int(x+2*w),int(y+2*h)),(0,255,0),2)
                #cv2.circle(frame, (int((x+x+w)/2),int((y+y+h)/2)),5,(0,0,255),5)
                if len(tracks)>0:
                    #cv2.putText(frame, tracks[0][0], (int(x-w),int(y-h-10)), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
                    
                    #['Frame','id','xmin','xmax','ymin','ymax']
                    data.append([frameCnt,tracks[0][0],int(x-w), int(x+2*w),int(y-h),int(y+2*h)])
            #Convert frames into png files
            cv2.imwrite(os.path.join(folderpath,str(frameCnt)+'.png'),frame)


        #Create video 
        #writer.write(cv2.resize(frame, (640,480)))

    #Release video object
    #video_stream.release()
    #writer.release()
except:
    pass

In [57]:
%cd Documents/Kitai/KITAI_v2.0

C:\Users\ccana\Documents\Kitai\KITAI_v2.0


In [72]:
with open('Data.csv', 'w') as creating_new_csv_file: 
   pass 
print("Empty File Created Successfully")

Empty File Created Successfully


In [13]:
file= open(os.path.join(folderpath,'Data.csv'), 'w') 
writer = writer(file)
writer.writerow(['Frame','id','xmin','xmax','ymin','ymax'])
writer.writerows(data)
file.close()